[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vcardonas/linkedin-project/blob/main/Cuadernos/2%20WebScraping_LinkedIn.ipynb#scrollTo=643905ba)

# **<center>WEB SCRAPING EN LINKEDIN</center>**
**<center> ¿Qué habilidades y herramientas son las más demandadas para el puesto de Analista de Datos en Colombia? </center>**

- ¿Cómo manipular el contenido obtenido para los perfiles de empleo? -> Silenium

## **LIBRERÍAS**

In [1]:
import pandas as pd
import time

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


## **CARGO: ANALISTA DE DATOS**

In [2]:
#Usar Brave chromedriver
driverPath = '/usr/local/Caskroom/chromedriver/107.0.5304.62/chromedriver'
binaryPath = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'

options = webdriver.ChromeOptions()
options.binary_location = binaryPath
driver = webdriver.Chrome(executable_path=driverPath, chrome_options=options)

# Abrir la página de linkedin
driver.get("https://www.linkedin.com/login")
 
# Esperar para que la página abra
time.sleep(5)

# Introducir email y contraseña
user_name = 'buwi_pjcdj53@tijux.com'
password = '1234567Vcs'
driver.find_element("xpath", "//*[@id='username']").send_keys(user_name)
driver.find_element("xpath", "//*[@id='password']").send_keys(password)

time.sleep(1)

# Dar clic en Inciar Sesión
driver.find_element("xpath", "//button[@type='submit']").click()
driver.implicitly_wait(30)


/var/folders/89/g3h9cy012t9b461ggllqnbw00000gn/T/ipykernel_4132/2470864465.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driverPath, chrome_options=options)
/var/folders/89/g3h9cy012t9b461ggllqnbw00000gn/T/ipykernel_4132/2470864465.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driverPath, chrome_options=options)


In [3]:
# Abrir link de empleos
url = 'https://www.linkedin.com/jobs/search/?currentJobId=3378218891&f_E=1%2C2%2C3&geoId=100876405&keywords=%22Analista%20de%20datos%22&location=Colombia&refresh=true'
driver.get(url)
time.sleep(1)

In [4]:
links = []

print("Links están siendo recolectados")

try: 
    for page in range(1, 28):
        time.sleep(2)
        jobs_block = driver.find_element(By.CLASS_NAME, "jobs-search-results-list") 
        jobs_list = jobs_block.find_elements(By.CSS_SELECTOR, ".jobs-search-results__list-item.occludable-update.p0.relative.scaffold-layout__list-item")

        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, "a")
            for a in all_links:
                if str(a.get_attribute("href")).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute("href") not in links: 
                    links.append(a.get_attribute("href"))
                else:
                    pass

            # Desplazar hacia abajo para ver todos los empleos
            driver.execute_script("arguments[0].scrollIntoView();", job)
        
        print(f"Recolectando los links en la página: {page}")
        
        # Ir a la siguiente página
        driver.find_element("xpath", f"//button[@aria-label='Página {page}']").click()
        time.sleep(3)
except:
    pass
print("Se encontraron " + str(len(links)) + " links para las ofertas de empleo")


Links están siendo recolectados
Recolectando los links en la página: 1
Recolectando los links en la página: 2
Recolectando los links en la página: 3
Recolectando los links en la página: 4
Recolectando los links en la página: 5
Recolectando los links en la página: 6
Recolectando los links en la página: 7
Recolectando los links en la página: 8
Recolectando los links en la página: 9
Recolectando los links en la página: 10
Recolectando los links en la página: 11
Recolectando los links en la página: 12
Recolectando los links en la página: 13
Recolectando los links en la página: 14
Recolectando los links en la página: 15
Recolectando los links en la página: 16
Recolectando los links en la página: 17
Recolectando los links en la página: 18
Recolectando los links en la página: 19
Recolectando los links en la página: 20
Recolectando los links en la página: 21
Recolectando los links en la página: 22
Se encontraron 434 links para las ofertas de empleo


In [5]:
# Crear listas vacías para almacenar información
titulos = []
empresas = []
ubicacion_empresa = []
modalidad_trabajo = []
fecha_publicacion = []
tipo_empleo = []
empleo_desc = []

i = 0
j = 1

# Visite cada link uno por uno para hacer scraping a la información
print('Visitar los links y recopilar información acaba de comenzar.')
for i in range(len(links)):
    try:
        driver.get(links[i])
        i = i + 1
        time.sleep(2)
        # Dar clic en Ver más
        driver.find_element(By.CLASS_NAME, "artdeco-card__actions").click()
        time.sleep(2)
    except:
        pass
    
    # Encontrar la información general de las ofertas de empleo
    contents = driver.find_elements(By.CLASS_NAME, 'p5')
    for content in contents:
        try:
            titulos.append(content.find_element(By.TAG_NAME, "h1").text)
            empresas.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__company-name").text)
            ubicacion_empresa.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__bullet").text)
            modalidad_trabajo.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__workplace-type").text)
            fecha_publicacion.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__posted-date").text)
            tipo_empleo.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__job-insight").text)
            
            print(f'Scraping the Job Offer {j} DONE.')
            j+= 1
            
        except:
            pass
        time.sleep(2)
        
    # Scraping la descripción de la oferta de empleo
    empleo_descripcion = driver.find_elements(By.CLASS_NAME, 'jobs-description__content')
    for descripcion in empleo_descripcion:
        texto = descripcion.find_element(By.CLASS_NAME, "jobs-box__html-content").text
        empleo_desc.append(texto)
        print(f'Scraping the Job Offer {j}')
        time.sleep(2)

Visitar los links y recopilar información acaba de comenzar.
Scraping the Job Offer 1 DONE.
Scraping the Job Offer 2
Scraping the Job Offer 2 DONE.
Scraping the Job Offer 3
Scraping the Job Offer 3 DONE.
Scraping the Job Offer 4
Scraping the Job Offer 4 DONE.
Scraping the Job Offer 5
Scraping the Job Offer 5 DONE.
Scraping the Job Offer 6
Scraping the Job Offer 6 DONE.
Scraping the Job Offer 7
Scraping the Job Offer 7
Scraping the Job Offer 7 DONE.
Scraping the Job Offer 8
Scraping the Job Offer 8 DONE.
Scraping the Job Offer 9
Scraping the Job Offer 9 DONE.
Scraping the Job Offer 10
Scraping the Job Offer 10 DONE.
Scraping the Job Offer 11
Scraping the Job Offer 11 DONE.
Scraping the Job Offer 12
Scraping the Job Offer 12 DONE.
Scraping the Job Offer 13
Scraping the Job Offer 13 DONE.
Scraping the Job Offer 14
Scraping the Job Offer 14
Scraping the Job Offer 14 DONE.
Scraping the Job Offer 15
Scraping the Job Offer 15 DONE.
Scraping the Job Offer 16
Scraping the Job Offer 16 DONE.
Scr

## **DATAFRAME Y EXPORTACIÓN**

In [6]:
# Crear dataframe
linkedin = pd.DataFrame(zip(titulos, empresas, ubicacion_empresa, modalidad_trabajo,
                            fecha_publicacion, tipo_empleo, empleo_desc),
                            columns =['Título', 'Empresa',
                                   'Ubicación','Modalidad de trabajo',
                                   'Publciación','Tipo de empleo', 'Descripción'])

linkedin

,Título,Empresa,Ubicación,Modalidad de trabajo,Publciación,Tipo de empleo,Descripción
0,Ruby on Rails Engineer,Aha!,"Bogotá, Distrito Capital, Colombia",En remoto,hace 12 minutos,Jornada completa · Sin experiencia,Acerca del empleo\nAha! is a very different ty...
1,DevOps Engineer - Remote - Colombia,FullStack Labs,"Cartagena de Indias, Bolívar, Colombia",En remoto,hace 5 días,Jornada completa · Sin experiencia,Acerca del empleo\nFullStack Labs is the faste...
2,"Product Developer, Retail Data Platform",Orium,Colombia,En remoto,hace 5 días,Jornada completa · Algo de responsabilidad,"Acerca del empleo\nSenior Product Developer, R..."
3,Trabajos remotos de Analista de Datos,Turing,Colombia,En remoto,hace 4 semanas,Jornada completa · Sin experiencia,Acerca del empleo\nUna empresa que cotiza en l...
4,Analista de datos y servicio al cliente,TH Plus Talento Humano,"Bogotá, Distrito Capital, Colombia",Presencial,hace 20 minutos,Jornada completa,"Acerca del empleo\nTH PLUS Talento Humano, req..."
...,...,...,...,...,...,...,...
379,DATA SCIENCE SR CONS - HT,NEORIS,"Santiago, Putumayo, Colombia",Híbrido,hace 5 días,Jornada completa · Algo de responsabilidad,"Acerca del empleo\nEn NEORIS, acelerador digit..."
380,QA (Quality Assurance) Automation Engineer - R...,FullStack Labs,"Cali, Valle del Cauca, Colombia",Híbrido,hace 3 semanas,Jornada completa · Algo de responsabilidad,Acerca del empleo\nFullStack Labs is the faste...
381,Analista de Seguridad - Bogota,DSM,Colombia,Híbrido,hace 1 día,Jornada completa · Prácticas,Acerca del empleo\nDSM – Bright Science. Brigh...
382,NoSQL Primary DBA,Datavail,Colombia,Híbrido,hace 2 semanas,Jornada completa · Algo de responsabilidad,Acerca del empleo\nJob Description\n\nDatavail...


In [7]:
# Revisión de descripción
linkedin['Descripción'].iloc[3]


'Acerca del empleo\nUna empresa que cotiza en la Bolsa de Nueva York y que ha revolucionado la industria farmacéutica durante más de un siglo con sus productos y servicios centrados en el cliente está buscando un analista de datos. El candidato seleccionado será responsable de revisar los datos para identificar información comercial clave y de explorar las formas de utilizarlos para resolver problemas. La compañía ha establecido con éxito una base de clientes global a través de sus servicios de investigación y desarrollo, fabricación y distribución de clase mundial. Esta es una oportunidad única para los desarrolladores que están ansiosos por trabajar con un gigante multinacional y acelerar sus carreras.\n\nResponsabilidades:\n\nAutomatizar los procesos comerciales existentes y los conjuntos de datos esenciales\nAnalizar y mejorar la escalabilidad y la eficiencia de los procedimientos de recopilación y almacenamiento de datos del sistema\nDiseñar código existente, crear servicios y con

In [9]:
# Almacenar los datos en un df
linkedin.to_csv('ws_linkedin.csv', index=False)